<img width="8%" alt="US Bureau of Labor Statistics.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/US%20Bureau%20of%20Labor%20Statistics.png" style="border-radius: 15%">

# US Bureau of Labor Statistics - Follow CPI
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=US+Bureau+of+Labor+Statistics+-+Follow+CPI:+Error+short+description">Bug report</a>

**Tags:** #inflation #us #BLS #naas #scheduler #asset #snippet #automation #ai #analytics

**Author:** [Jeremy Ravenel](https://www.linkedin.com/in/j%C3%A9r%C3%A9my-ravenel-8a396910/)

**Last update:** 2023-04-12 (Created: 2022-07-16)

**Description:** This notebook provides an analysis of the US Bureau of Labor Statistics Consumer Price Index (CPI) over time.

## Input

### Setup libraries

In [ ]:
# Install CPI Python Package
!pip install cpi

In [2]:
import pandas as pd
import cpi
import seaborn as sns
import matplotlib.pyplot as plt
import naas
from naas_drivers import plotly

## Model

### Update CPI latest data
BLS is constantly updating with the latest inflation information

In [3]:
# this can take some time
cpi.update()

### Get monthly data

In [4]:
cpi_items_df = cpi.series.get(seasonally_adjusted=False).to_dataframe()
cpi_items_df = cpi_items_df[cpi_items_df["period_type"] == "monthly"]
cpi_items_df["date"] = pd.to_datetime(cpi_items_df["date"])
cpi_items_df = cpi_items_df.set_index("date")

### Show dataframe

In [5]:
df = cpi_items_df
df.head()

### Create plot

In [6]:
fig = plt.figure(dpi=200)
df["value"].loc["2010":"2023"].plot(figsize=(10, 4))
plt.title("Evolution of CPI")
plt.xlabel("Date")
plt.ylabel("CPI Value")

### Simulate 2% inflation YOY

In [10]:
# Starting value
cpi_items_df["value"].loc["2010":"2023"].iloc[0]

start = cpi_items_df["value"].loc["2010":"2023"].iloc[0]

periods = len(cpi_items_df["value"].loc["2010":"2023"]) // 12

In [12]:
def get_target_cpi(previous_cpi):
    return previous_cpi + 0.02 * (previous_cpi)

In [13]:
target_cpis = [start]
for year in range(0, periods):
    target_cpis.append(get_target_cpi(target_cpis[year]))

target_cpis

In [14]:
dates = pd.date_range("2010-01-01", "2023-01-01", periods=periods + 1)
target_cpi_series = pd.Series(data=target_cpis, index=dates)

In [22]:
fig = plt.figure(dpi=200)
cpi_items_df["value"].loc["2010":"2023"].plot(figsize=(10, 6))
target_cpi_series.plot(ls="--")
plt.title("Evolution of CPI vs a 2% inflation rate yearly")
plt.xlabel("Date")
plt.ylabel("CPI Value")

## Output

### Save result in png

In [20]:
plt.savefig("CPI.png")

### Save result in csv

In [8]:
df.to_csv("CPI.csv", index=False)

### Share your output with naas

In [9]:
naas.asset.add("CPI.csv")
naas.asset.add("CPI.png")

# -> Uncomment the line below to remove your asset
# naas.asset.delete()